In [ ]:
# Install necessary libraries
! pip install datasets
! pip install huggingface_hub
# Connect to hugging face
! huggingface-cli login --token YOUR_HF_TOKEN

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Merge two datasets

In [ ]:
from datasets import load_dataset

In [ ]:
dataset_2 = load_dataset("bebeyondo/medical-subsuming")
dataset_3 = load_dataset("bebeyondo/medical-adjacent")

README.md:   0%|          | 0.00/617 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.9M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26418 [00:00<?, ? examples/s]

...

val-00000-of-00001.parquet:   0%|          | 0.00/4.76M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.75M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26418 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/8807 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8807 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets
# Concatenate the datasets
merged_dataset = concatenate_datasets([dataset_2['train'], dataset_3['train']])
merged_val_dataset = concatenate_datasets([dataset_2['val'], dataset_3['val']])
merged_test_dataset = concatenate_datasets([dataset_2['test'], dataset_3['test']])


In [ ]:
merged_dataset = merged_dataset.shuffle(seed=42)
merged_val_dataset = merged_val_dataset.shuffle(seed=42)
merged_test_dataset = merged_test_dataset.shuffle(seed=42)

In [ ]:
from datasets import DatasetDict

In [ ]:
dataset = DatasetDict({
    'train': merged_dataset,
    'val': merged_val_dataset,
    'test': merged_test_dataset
})

In [ ]:
dataset = dataset.remove_columns("pairs_unique_id")

In [ ]:
dataset["train"][7]

{'idx': 36366,
 'anchor': 'on the ribs in the submuscular position. Both the bar and the stabilizer were fixed together onto the underlying rib with multiple interrupted non-absorbable sutures. An excision of the entire soft tissue mass was carried out right after the Nuss procedure. The visible tumor was of grayish color and of a gelatine-like consistency, stating immediately below a layer of subepidermal fat and did not extend deep to the pectoralis major. A 21.0 × 17.5 × 5.0',
 'positive': 'cm mass was successfully resected. Signs of erosion of the rib and sternum were no found. After excision of the bulky skin, two Redon drain was inserted and the wound was closed without tension. The patient received an epidural catheter for perioperative patient-controlled analgesia (PCA) for 4 days, and had a good postoperative recovery without complications. He and his parents expressed satisfaction with the cosmetic relief which achieved by the resection and'}

In [ ]:
dataset = concatenate_datasets([merged_dataset, merged_val_dataset, merged_test_dataset])
dataset = dataset.shuffle(seed=42)

In [ ]:
dataset

Dataset({
    features: ['pairs_unique_id', 'idx', 'anchor', 'positive'],
    num_rows: 88064
})

# hard negatives

In [ ]:
import random  # Import the random module to sample negative anchors

# Function to create a hard negative sample
def create_hard_negatives(example, grouped_by_idx):
    current_anchor = example['anchor']
    idx = example['idx']

    # Get a different anchor from the same idx
    candidates = [pair[0] for pair in grouped_by_idx[idx] if pair[0] != current_anchor]

    negative_anchor = random.choice(candidates)

    # Return the example with the new "negative" column
    return {
        'idx': example['idx'],
        'anchor': example['anchor'],
        'positive': example['positive'],
        'negative': negative_anchor
    }

# Function to process the entire dataset
def process_dataset(dataset):
    # First, group by idx to ensure we get negatives from the same idx
    grouped_by_idx = {}

    # Create a dictionary where the key is idx and the value is a list of (anchor, positive) pairs
    for example in dataset:
        idx = example['idx']
        if idx not in grouped_by_idx:
            grouped_by_idx[idx] = []
        grouped_by_idx[idx].append((example['anchor'], example['positive']))

    # Apply the function to each example in the dataset
    return dataset.map(lambda x: create_hard_negatives(x, grouped_by_idx))


In [ ]:
dataset = process_dataset(dataset)

Map:   0%|          | 0/88064 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

train_valid_split = train_test_split['train'].train_test_split(test_size=0.25, seed=42)

dataset = DatasetDict({
    'train': train_valid_split['train'],
    'val': train_valid_split['test'],
    'test': train_test_split['test']
})

# Push to Hugging face

In [ ]:
!git config --global credential.helper store

In [ ]:
dataset.push_to_hub("bebeyondo/medical-triplet")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/53 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/bebeyondo/medical-triplet/commit/d222c0ccc67b7787cf331179eeb7981816960c64', commit_message='Upload dataset', commit_description='', oid='d222c0ccc67b7787cf331179eeb7981816960c64', pr_url=None, pr_revision=None, pr_num=None)